In [ ]:
class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, date, path=None, N=31, local=False):
        '''
        '''
        
        self.date = GPSDateTime(date)
        if not path:
            path = os.path.join(localpath, 'almanac')
        self.path = path
        if not os.path.exists(path):
            os.makedirs(path)
        for i in range(N):
            idate = self.date - timedelta(i)
            idoy = idate.doy
            fname = '%d%03d.ALM' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            fname = '%d%03d.alm' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            if not local:
                fname = self.download(idate)
                if fname:
                    break
            fname = False
        if not fname:
            raise RuntimeError('No almanac file found for %s in %s' \
                               % (self.date.date(), path))
        else:
            fid = open_file(os.path.join(path, fname))
            while True:
                try:
                    line = fid.next()
                except StopIteration:
                    break
                if not line: continue
                if line[:2]=='**':
                    ID = 'G%02d' % int(fid.next().split(':')[1])
                    self[ID]={}
                    self[ID]['HEALTH'] = float(fid.next().split(':')[1])
                    self[ID]['ECC']    = float(fid.next().split(':')[1])
                    self[ID]['TOA']    = float(fid.next().split(':')[1])
                    self[ID]['INC']    = float(fid.next().split(':')[1])
                    self[ID]['RRAND']  = float(fid.next().split(':')[1])
                    self[ID]['SMA']    = float(fid.next().split(':')[1])
                    self[ID]['RAND']   = float(fid.next().split(':')[1])
                    self[ID]['AOP']    = float(fid.next().split(':')[1])
                    self[ID]['AM']     = float(fid.next().split(':')[1])
                    self[ID]['AF0']    = float(fid.next().split(':')[1])
                    self[ID]['AF1']    = float(fid.next().split(':')[1])
                    week = int(fid.next().split(':')[1])
                    realweek = idate.GPSweek
                    if week<realweek:
                        week+=1024*(realweek/1024)
                    self[ID]['WK'] = week

    def download(self, date):
        '''
        '''
        toa_1 = ['', '380928', '466944', '552960', '032768', '118784', '']
        toa_2 = ['319488', '405504', '503808', '589824', '061440',
                 '147456', '233472']

        if date<datetime(1996,3,31): toa = toa_1
        else: toa = toa_2

        URL1 = 'http://www.navcen.uscg.gov/?Do=getAlmanac&almanac=%03d&year=%d&format=yuma' \
                % (date.doy, date.year)
        URL2 = 'http://celestrak.com/GPS/almanac/Yuma/%d/almanac.yuma.week%04d.%s.txt' \
                % (date.year, (date.GPSweek)%1024, toa[date.GPSdow])
        URL3 = 'https://gps.afspc.af.mil/gps/archive/%d/almanacs/yuma/wk%d%s.alm' \
                % (date.year, date.GPSweek, date.strftime('%a').lower())
        urls = [URL1, URL2, URL3]

        for url in urls:
            err = True
            try:
                print 'Downloading almanac file from %s' % url
                data = urlopen(url)
                line = data.readline()
                if '****' in line and 'Week' in line:
                    err = False
                    break
            except:
                err = True

        if err: return False
        fname = '%d%03d.ALM' % (date.year, date.doy)
        new_alm = open(os.path.join(self.path, fname), 'w')
        new_alm.write(line+data.read())
        new_alm.close()
        data.close()
        return fname
    def sat_pos(self, prns, dt, user_pos=(0, 0, 0)):
        '''
        Translate from gps.c of "The Essential GNSS Project"
        Copyright (c) 2007 author: doxygen
        '''
        
        # iterate to include the Sagnac correction
        # since the range is unknown, an approximate of 70 ms is good enough
        # to start the iterations so that 2 iterations are enough

        ret   = []
        rango = 0.07*LIGHT_SPEED
        userX, userY, userZ = user_pos
        for prn in prns:
            for i in xrange(2):
                Xs, Ys, Zs = self.calc_sat_pos(prn, dt, rango)
                dx         = Xs - userX
                dy         = Ys - userY
                dz         = Zs - userZ
                rango      = math.sqrt( dx*dx + dy*dy + dz*dz )
            ret.append([Xs, Ys, Zs])

        return ret

    def calc_sat_pos(self, prn, dt, estimate_range):
        '''
        '''
        
        toe = self[prn]['TOA']
        m0  = self[prn]['AM']
        delta_n = 0
        tot = dt.GPSweek*SECONDS_IN_WEEK+dt.GPStow
        tk  = tot - (self[prn]['WK']*SECONDS_IN_WEEK+toe)
        # compute the corrected mean motion
        a   = self[prn]['SMA']**2
        n   = math.sqrt(GPS_GRAVITY_CONSTANT/(a*a*a))
        n  += delta_n
        # kepler's equation for eccentric anomaly
        M = m0 + n*tk
        E = M
        for i in range(8):
            E = M + self[prn]['ECC']*math.sin(E)

        cosE = math.cos(E)
        sinE = math.sin(E)
        # true anomaly
        v = math.atan2(math.sqrt(1.0-self[prn]['ECC']**2)*sinE, (cosE - self[prn]['ECC']))
        #argument of latitude
        u = v + self[prn]['AOP']
        # radius in orbital plane
        r = a * (1.0 - self[prn]['ECC']*math.cos(E))
        # orbital inclination
        i = self[prn]['INC']

        #argument of latitude correction
        #! no correction
        cosu = math.cos(u)
        sinu = math.sin(u)
        x_op = r*cosu
        y_op = r*sinu

        omegak = self[prn]['RAND'] + (self[prn]['RRAND'] - GPS_WGS84_EARTH_ROTATION_RATE)*tk - GPS_WGS84_EARTH_ROTATION_RATE*(toe + estimate_range/LIGHT_SPEED )

        # compute the WGS84 ECEF coordinates,
        # vector r with components x & y is now rotated using, R3(-omegak)*R1(-i)
        cos_omegak = math.cos(omegak)
        sin_omegak = math.sin(omegak)
        cosi = math.cos(i)
        sini = math.sin(i)
        x = x_op * cos_omegak - y_op * sin_omegak * cosi
        y = x_op * sin_omegak + y_op * cos_omegak * cosi
        z = y_op * sini

        return x,y,z
        def get_sat_clock_correction(self, prns, dt):
        '''
        '''
        ret = []
        for prn in prns:
            toe = self[prn]['TOA']
            toc = self[prn]['TOA']
            m0  = self[prn]['AM']
            delta_n = 0
            tot = dt.GPSweek*SECONDS_IN_WEEK+dt.GPStow
            tk  = tot - (self[prn]['WK']*SECONDS_IN_WEEK+toe)
            tc  = tot - (self[prn]['WK']*SECONDS_IN_WEEK + toc)
            # compute the corrected mean motion
            a   = self[prn]['SMA']**2
            n   = math.sqrt(GPS_GRAVITY_CONSTANT/(a*a*a))
            n  += delta_n
            # kepler's equation for eccentric anomaly
            M = m0 + n*tk
            E = M
            for i in xrange(8):
                E = M + self[prn]['ECC']*math.sin(E)

            #relativistic correction
            d_tr  = GPS_CLOCK_CORRECTION_RELATIVISTIC_CONSTANT*self[prn]['ECC']*self[prn]['SMA']*math.sin(E)
            d_tr *= LIGHT_SPEED

            #clock correction
            d_tsv  = self[prn]['AF0']+self[prn]['AF1']*tc#+af2*tc*tc
            ret.append(d_tsv*LIGHT_SPEED+d_tr)

            #clock drift
            #clock_drift = self[prn]['AF1']*LIGHT_SPEED

        return ret

class gpselaz(dict):
    '''
    Class to calculate el, az, lat and lon of satellites, uses yuma almanac info
    '''
    def __init__(self, date, path=None):
        yuma = Yuma(date, path)
        U = 398603.2
        self.date = GPSDateTime(date)
        self.elemnts = {}
        for id in yuma:
            IDate = GPSDateTime(GPSweek=yuma[id]['WK'])
            ragren = self.ra_gren(IDate)
            SMA = yuma[id]['SMA']*yuma[id]['SMA']/1000.0
            #***********************************************************
            #*    CALCULATE PERIOD IN MINUTES FROM SMA                 *
            #*    FORMULA   SMA=(U**1/3)*((86400.0/TWO*PI*N)**2/3)     *
            #*              SMA=SEMI-MAJOR AXIS(KM)                    *
            #*              U=398603.2                                 *
            #*              N=PERIOD(REVS/DAY)                         *
            #***********************************************************
            N = 86400.0/(2*PI)*math.sqrt(U/SMA**3)
            N = 1440.0/N
            #calculate argument of perigee
            AOP = fmod(yuma[id]['AOP'])*rad2deg
            #calculate inclination
            INC = fmod(yuma[id]['INC'])*rad2deg
            RAAN = (yuma[id]['RAND']*rad2deg + ragren)*deg2rad
            RAAN = fmod(RAAN)*rad2deg
            #calculate mean anomaly at 00 hrs
            XMO = 1440.0/N*360.0/86400.0/rad2deg*yuma[id]['TOA']
            AM = fmod(yuma[id]['AM'] - XMO)*rad2deg
            self.elemnts[id] = [IDate, SMA, N, yuma[id]['ECC'], INC, AM, AOP,
                                RAAN]


In [2]:
class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, date, path=None, N=31, local=False):
        '''
        '''
        
        self.date = GPSDateTime(date)
        if not path:
            path = os.path.join(localpath, 'almanac')
        self.path = path
        if not os.path.exists(path):
            os.makedirs(path)
        for i in range(N):
            idate = self.date - timedelta(i)
            idoy = idate.doy
            fname = '%d%03d.ALM' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            fname = '%d%03d.alm' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            if not local:
                fname = self.download(idate)
                if fname:
                    break
            fname = False
        if not fname:
            raise RuntimeError('No almanac file found for %s in %s' \
                               % (self.date.date(), path))
        else:
            fid = open_file(os.path.join(path, fname))
            while True:
                try:
                    line = fid.next()
                except StopIteration:
                    break
                if not line: continue
                if line[:2]=='**':
                    ID = 'G%02d' % int(fid.next().split(':')[1])
                    self[ID]={}
                    self[ID]['HEALTH'] = float(fid.next().split(':')[1])
                    self[ID]['ECC']    = float(fid.next().split(':')[1])
                    self[ID]['TOA']    = float(fid.next().split(':')[1])
                    self[ID]['INC']    = float(fid.next().split(':')[1])
                    self[ID]['RRAND']  = float(fid.next().split(':')[1])
                    self[ID]['SMA']    = float(fid.next().split(':')[1])
                    self[ID]['RAND']   = float(fid.next().split(':')[1])
                    self[ID]['AOP']    = float(fid.next().split(':')[1])
                    self[ID]['AM']     = float(fid.next().split(':')[1])
                    self[ID]['AF0']    = float(fid.next().split(':')[1])
                    self[ID]['AF1']    = float(fid.next().split(':')[1])
                    week = int(fid.next().split(':')[1])
                    realweek = idate.GPSweek
                    if week<realweek:
                        week+=1024*(realweek/1024)
                    self[ID]['WK'] = week


In [65]:
'''
Module containing a UTC-based datetime class with additional gps time constants.
'''

import os
import re
import time
from datetime import datetime, timedelta, date
from math import fmod

GPS_EPOCH_JDAY = 2444245
TIMESTAMP0 = datetime(1970, 1, 1, 0, 0)
GPSTIMESTAMP0 = datetime(1980, 1, 6, 0, 0)

class GPSDateTime(object):
    '''
    A UTC-based datetime object.

    This datetime class is based on the POSIX time, a system for describing
    instants in time, defined as the number of seconds elapsed since midnight
    Coordinated Universal Time (UTC) of Thursday, January 1, 1970. Using a
    single float timestamp allows higher precision as the default Python
    :class:`datetime.datetime` class. It features additional string patterns 
    during object initialization and GPS time constants.
    
    '''
    
    timestamp = 0.0
    DEFAULT_PRECISION = 6
    
    def __init__(self, *args, **kwargs):
        '''
        Creates a new GPSDateTime object.
        '''
        self.precision = kwargs.pop('precision', self.DEFAULT_PRECISION)
        filename = kwargs.pop('filename', False)
        if len(args) == 0 and len(kwargs) == 0:
            self.timestamp = time.time()
            return
        elif len(args) == 1:
            value = args[0]
            # check types
            try:
                # got a timestamp
                self.timestamp = value.__float__()
                return
            except:
                pass
            if isinstance(value, datetime):
                return self.from_datetime(value)
            elif isinstance(value, date):
                dt = datetime(value.year, value.month, value.day)
                return self.from_datetime(dt)
            elif isinstance(value, basestring):
                value = value.strip()
                if filename is True:
                    value = ''.join([n for n in re.findall(r'\d+', value) if len(n)>1])
                elif isinstance(filename, basestring) and filename.lower() == 'rinex':
                    value = value[4:7]+value[9:11]
                # try to apply some standard patterns
                value = value.replace('T', ' ')
                value = value.replace('_', ' ')
                value = value.replace('-', ' ')
                value = value.replace(':', ' ')
                value = value.replace('/', ' ')
                value = value.replace(',', ' ')
                value = value.replace('Z', ' ')
                value = value.replace('W', ' ')
                # check for ordinal date (julian date)
                parts = [val for val in value.split(' ') if val]
                # check for patterns                
                if isinstance(filename, basestring) and filename.lower() == 'rinex':
                    pattern = "%j%y"
                elif len(parts) == 1 and len(value) == 7 and value.isdigit():
                    pattern = "%Y%j"
                elif len(parts) == 2 and len(parts[1]) == 3 and parts[1].isdigit():                    
                    value = ''.join(parts)
                    pattern = "%Y%j"
                elif len(parts) == 2 and len(parts[0]) == 7 and len(parts[1]) == 6:                    
                    value = ''.join(parts)
                    pattern = "%Y%j%H%M%S"
                elif len(parts) == 3 and len(parts[1]) == 3 and len(parts[2]) == 6:                    
                    value = ''.join(parts)
                    pattern = "%Y%j%H%M%S"
                else:
                    # some parts should have 2 digits
                    for i in range(1, min(len(parts), 6)):
                        if len(parts[i]) == 1:
                            parts[i] = '0' + parts[i]
                    # standard date string
                    value = ''.join(parts)
                    if len(value) == 6:
                        pattern = "%y%m%d"
                    elif len(value) == 10:
                        pattern = "%y%m%d%H%M"
                    elif len(value) == 12:
                        pattern = "%y%m%d%H%M%S"
                    elif len(value) == 13:
                        pattern = "%Y%j%H%M%S"
                    elif len(value) == 14:
                        pattern = "%Y%m%d%H%M%S"
                    else:
                        pattern = "%Y%m%d"
                ms = 0
                if '.' in value:
                    parts = value.split('.')
                    value = parts[0].strip()
                    try:
                        ms = float('.' + parts[1].strip())
                    except:
                        pass
                # all parts should be digits now - here we filter unknown
                # patterns and pass it directly to Python's  datetime
                if not ''.join(parts).isdigit():
                    dt = datetime(*args, **kwargs)
                    self.from_datetime(dt)
                    return                
                
                dt = datetime.strptime(value, pattern)
                self.from_datetime(dt, ms)
                return
        # check new kwargs
        if 'julday' in kwargs:
            jd     = kwargs.pop('julday')
            a      = int(jd+0.5)
            b      = a+1537
            c      = int((b-122.1)/365.25)
            d      = int(365.25*c)
            e      = int((b-d)/30.6001)
            td     = b-d-int(30.6001*e)+fmod(jd+0.5, 1.0)
            day    = int(td)
            td    -= day
            td    *= 24.
            hour   = int(td)
            td    -= hour
            td    *= 60.
            minute = int(td)
            td    -= minute
            td    *= 60.
            second = int(td)
            month  = (e-1-12*int(e/14))
            year   = (c-4715-int((7+month)/10.))
            
            kwargs['year']   = year
            kwargs['month']  = month
            kwargs['day']    = day
            kwargs['hour']   = hour
            kwargs['minute'] = minute
            kwargs['second'] = second

        if 'doy' in kwargs and 'year' in kwargs:
            year  = kwargs['year']
            day   = kwargs.pop('doy')
            month = 0
            while day>0:
                nd     = GPSDateTime().days_in_month(year, month+1)
                day   -= nd
                month += 1
            day += nd
            kwargs['month'] = month
            kwargs['day']   = day

        if 'GPSweek' in kwargs:
            week = kwargs.pop('GPSweek')
            sow  = kwargs.pop('GPSsow', 0)
            dt   = GPSTIMESTAMP0 + timedelta(days=7*week, seconds=sow)
            self.from_datetime(dt)
            return
        # check if seconds are given as float value
        if len(args) == 6 and isinstance(args[5], float):
            kwargs['microsecond'] = int(args[5] % 1 * 1000000)
            kwargs['second']      = int(args[5])
            args                  = args[0:5]
        dt = datetime(*args, **kwargs)
        self.from_datetime(dt)        

    def from_datetime(self, dt, ms=0.0):
        '''
        Set timestamp from a datetime object
        '''
        try:
            td = (dt - TIMESTAMP0)
        except TypeError:
            td = (dt.replace(tzinfo=None) - dt.utcoffset()) - TIMESTAMP0
        self.timestamp = (td.microseconds + (td.seconds + td.days * 86400) *
                          1000000) / 1000000.0 +ms    
    
    def _set(self, **kwargs):
        '''
        Sets current timestamp using kwargs.
        '''
        year = kwargs.get('year', self.year)
        month = kwargs.get('month', self.month)
        day = kwargs.get('day', self.day)
        hour = kwargs.get('hour', self.hour)
        minute = kwargs.get('minute', self.minute)
        second = kwargs.get('second', self.second)
        microsecond = kwargs.get('microsecond', self.microsecond)
        doy = kwargs.get('doy', None)
        if doy:
            self.timestamp = GPSDateTime(year=year, doy=doy, hour=hour,
                                         minute=minute, second=second,
                                         microsecond=microsecond).timestamp
        else:
            self.timestamp = GPSDateTime(year, month, day, hour, minute,
                                         second, microsecond).timestamp
    
    def get_timestamp(self):
        return self.timestamp


    def get_datetime(self):
        return TIMESTAMP0 + timedelta(seconds=self.timestamp)

    datetime = property(get_datetime)

    def get_year(self):
        return self.get_datetime().year
    
    def set_year(self, value):
        self._set(year=value)

    year = property(get_year, set_year)
    
    def get_month(self):
        return self.get_datetime().month
    
    def set_month(self, value):
        self._set(month=value)

    month = property(get_month, set_month)
    
    def get_day(self):
        return self.get_datetime().day
    
    def set_day(self, value):
        self._set(day=value)

    day = property(get_day, set_day)
    
    def get_hour(self):
        return self.get_datetime().hour
    
    def set_hour(self, value):
        self._set(hour=value)

    hour = property(get_hour, set_hour)
    
    def get_minute(self):
        return self.get_datetime().minute
    
    def set_minute(self, value):
        self._set(minute=value)

    minute = property(get_minute, set_minute)
    
    def get_second(self):
        return self.get_datetime().second
    
    def set_second(self, value):
        self._set(second=value)

    second = property(get_second, set_second)
    
    def get_microsecond(self):
        return self.get_datetime().microsecond
    
    def set_microsecond(self, value):
        self._set(microsecond=value)

    microsecond = property(get_microsecond, set_microsecond)
    
    def get_doy(self):
        '''
        Returns the day of the year of the current GPSDateTime object.
        '''        
        return self.utctimetuple().tm_yday
    
    def set_doy(self, value):
        '''
        Set the day of the year of the current GPSDateTime object.
        '''
        self._set(doy=value)
    
    doy = property(get_doy, set_doy)

    def get_julday(self):
        '''
        Returns the Julian day of the current GPSDateTime object.
        '''
        yy, mm, dd     = self.day, self.month, self.year
        hour, minute, second = self.hour, self.minute, self.second
        
        
        a = (14 - self.month)//12
        y = self.year + 4800 - a
        m = self.month + 12*a - 3
        return self.day + ((153*m + 2)//5) + 365*y + y//4 - y//100 + y//400 - 32045

        
        
        return dd-32075+1461*(yy+4800+(mm-14)/12)/4+ \
                367*(mm-2-(mm-14)/12*12)/12-3*((yy+4900+(mm-14)/12)/100)/4
        

        a = (14 - self.month)//12
        y = self.year + 4800 - a
        m = self.month + 12*a - 3
        jd = self.day + ((153*m + 2)//5) + 365*y + y//4 - y//100 + y//400 - 32045

        if self.month<= 2:
            y = self.year - 1
            m = self.month + 12
        else:
            y = self.year
            m = self.month
        jd1 = int(365.25*y)+int(30.6001*(m+1.))+self.day+self.hour/24.+self.minute/1440.+self.second/86400.+1720981.5

        return jd1

    julday = property(get_julday)

    def get_GPSweek(self):
        """
        Returns the GPS Week of the current GPSDateTime object.
        """
        if self >= GPSTIMESTAMP0:
            return (self.datetime-GPSTIMESTAMP0).days/7
        else:
            return 0
        

    GPSweek = property(get_GPSweek)

    def get_GPSdow(self):
        """
        Returns the day of the Week number of the current GPSDateTime object.
        """
        
        return int((self.julday%7+1)%7)

    GPSdow  = property(get_GPSdow)

    def get_GPStow(self):
        """
        Returns the seconds of the Week of the current GPSDateTime object.
        """
        
        return self-GPSDateTime(GPSweek=self.GPSweek)

    GPStow  = property(get_GPStow)

    def get_doys(self):
        """
        Returns a decimal day of the year of the current GPSDateTime object.
        """
        return self.doy+self.hours/24.

    doys = property(get_doys)

    def get_hours(self):
        """
        Returns a decimal hours of the current GPSDateTime object.
        """
        return self.hour+self.minute/60.+self.second/3600.

    hours = property(get_hours)

    def get_seconds(self):
        """
        Returns total of seconds of the current GPSDateTime object.
        """
        return self.hour*3600+self.minute*60+self.second

    seconds = property(get_seconds)

    def __add__(self, value):
        '''
        Adds seconds and microseconds to current GPSDateTime object.
        '''
        if isinstance(value, timedelta):
            # see timedelta.total_seconds
            value = (value.microseconds + (value.seconds + value.days *
                     86400) * 1000000) / 1000000.0
        return GPSDateTime(self.timestamp + value)

    def __sub__(self, value):
        '''
        Subtracts seconds and microseconds from current GPSDateTime object.
        '''
        if isinstance(value, (GPSDateTime, datetime)):
            return round(self.timestamp - GPSDateTime(value).timestamp, self.precision)
        elif isinstance(value, timedelta):
            # see timedelta.total_seconds
            value = (value.microseconds + (value.seconds + value.days *
                     86400) * 1000000) / 1000000.0
        return GPSDateTime(self.timestamp - value)

    def __float__(self):
        return self.timestamp
    
    def __int__(self):
        return int(self.timestamp)
    
    def __eq__(self, other):
        '''
        Rich comparison operator '=='
        '''
        try:
            return round(self.timestamp - GPSDateTime(other).timestamp, self.precision) == 0
        except (TypeError, ValueError):
            return False

    def __ne__(self, other):
        '''
        Rich comparison operator '!='
        '''
        return not self.__eq__(other)

    def __lt__(self, other):
        '''
        Rich comparison operator '<'.
        '''
        try:
            return round(self.timestamp - GPSDateTime(other).timestamp, self.precision) < 0
        except (TypeError, ValueError):
            return False

    def __le__(self, other):
        '''
        Rich comparison operator '<='.
        '''
        try:
            return round(self.timestamp - GPSDateTime(other).timestamp, self.precision) <= 0
        except (TypeError, ValueError):
            return False

    def __gt__(self, other):
        '''
        Rich comparison operator '>'
        '''
        try:
            return round(self.timestamp - GPSDateTime(other).timestamp, self.precision) > 0
        except (TypeError, ValueError):
            return False

    def __ge__(self, other):
        '''
        Rich comparison operator '>='
        '''
        try:
            return round(self.timestamp - GPSDateTime(other).timestamp, self.precision) >= 0
        except (TypeError, ValueError):
            return False
    
    def __unicode__(self):
        return str(self.__str__())
    
    def __str__(self):
        return self.strftime('%Y-%m-%d %H:%M:%S.%fUTC')    
    
    def __repr__(self):
        return 'GPSDateTime' + self.get_datetime().__repr__()[17:]
    
    def __hash__(self):
        return hash(self.timestamp)
    
    def __abs__(self):
        return abs(self.timestamp)
    
    def replace(self, **kwargs):
        return GPSDateTime(self.datetime.replace(**kwargs))
    
    def date(self):
        return GPSDateTime(datetime(self.year, self.month, self.day))
    
    def strftime(self, fmt):
        return self.get_datetime().strftime(fmt)
    
    @staticmethod
    def strptime(s, fmt):
        return GPSDateTime(datetime.strptime(s, fmt))
    
    def timetz(self):        
        return self.get_datetime().timetz()
    
    def utcoffset(self):
        return self.get_datetime().utcoffset()
    
    def dst(self):
        return self.get_datetime().dst()
    
    def tzname(self):
        return self.get_datetime().tzname()
    
    def ctime(self):
        return self.get_datetime().ctime()
    
    def isoweekday(self):
        return self.get_datetime().isoweekday()
    
    def isocalendar(self):
        return self.get_datetime().isocalendar()
    
    def isoformat(self, sep="T"):
        return self.get_datetime().isoformat(sep=sep)
    
    def timetuple(self):
        return self.get_datetime().timetuple()

    def utctimetuple(self):        
        return self.get_datetime().utctimetuple()
    
    @staticmethod
    def today():
        dt = datetime.now()
        return GPSDateTime(dt.year, dt.month, dt.day)
    
    @staticmethod    
    def now():       
        return GPSDateTime()
        
    @staticmethod
    def utcnow():      
        return GPSDateTime()
    
    def toordinal(self):       
        return self.get_datetime().toordinal()

    def get_str(self, all=False):
        """
        Returns common string representation.
        """
        if all:
            return self.strftime('%y%m%d_%H%M%S')
        else:
            return self.strftime('%y%m%d')

    def is_leap_year(self, year=None):
        '''
        Returns True if year is a leap year, False otherwise.
        '''
        if not year: year = self.year
        if year%4 == 0:
            if year%100 == 0:
                if year%400 == 0:
                    return True
                else:
                    return False
            else:
                return True
        else:
            return False

    def days_in_month(self, year=None, month=None):
        '''
        Return the number of days in month for the given month and year
        '''
        if not year : year  = self.year
        if not month: month = self.month
        if month == 2:
            if GPSDateTime().is_leap_year(year):
                return 29
            else:
                return 28
        elif month in (1, 3, 5, 7, 8, 10, 12):
            return 31
        else:
            return 30


if __name__ == '__main__':
    dt = GPSDateTime(2014,12,28)
    print ('Date           :', dt)
    print ('Julday         :', dt.julday)
    print ('Day of the year:', dt.doy)
    print ('GPS week       :', dt.GPSweek)
    print ('Day of week    :', dt.GPSdow)
    print ('GPS seconds    :', dt.GPStow)
    print ('Timestamp      :', dt.timestamp)
    

Date           : 2014-12-28 00:00:00.000000UTC
Julday         : 2457020
Day of the year: 362
GPS week       : 1825.0
Day of week    : 0
GPS seconds    : 0.0
Timestamp      : 1419724800.0


In [59]:
x=GPSDateTime(date)
print(x)

NameError: name 'basestring' is not defined

In [31]:
class MiClase:
    """Simple clase de ejemplo"""
    i = 12345
    def f(self):
        return 'hola mundo'
x=MiClase
print(x.i)
print(x.f)

12345
<function MiClase.f at 0x7f8dd89961e0>


In [32]:
x = MiClase()
def __init__(self):
    self.datos = []

In [47]:
xf = x.f
while True:
    print(xf)

TypeError: f() missing 1 required positional argument: 'self'

In [36]:
from urllib.request import urlopen
url='http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt'
print ('Downloading almanac file from %s' % url)
data = urlopen(url)
for line in data:
   print(line)
line1 = data.readline()
print(line1,"gg")

b'******** Week 1011 almanac for PRN-01 ********\r\n'
b'ID:                         01\r\n'
b'Health:                     000\r\n'
b'Eccentricity:               0.8257865906E-002\r\n'
b'Time of Applicability(s):  61440.0000\r\n'
b'Orbital Inclination(rad):   0.9742300000\r\n'
b'Rate of Right Ascen(r/s):  -0.7748894201E-008\r\n'
b'SQRT(A)  (m 1/2):           5153.620117\r\n'
b'Right Ascen at Week(rad):  -0.4721338949E+000\r\n'
b'Argument of Perigee(rad):   0.690804055\r\n'
b'Mean Anom(rad):             0.8929457212E+000\r\n'
b'Af0(s):                    -0.1411437988E-003\r\n'
b'Af1(s/s):                  -0.7275957614E-011\r\n'
b'week:                        1011\r\n'
b'\r\n'
b'******** Week 1011 almanac for PRN-02 ********\r\n'
b'ID:                         02\r\n'
b'Health:                     000\r\n'
b'Eccentricity:               0.1872158051E-001\r\n'
b'Time of Applicability(s):  61440.0000\r\n'
b'Orbital Inclination(rad):   0.9532935591\r\n'
b'Rate of Right Ascen(r/s):  -0.795461

In [3]:
from urllib.request import urlopen

link = 'http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt'
f = urlopen(link)
myfile = f.read()
lines = myfile.next()
print(myfile)

AttributeError: 'bytes' object has no attribute 'next'

In [45]:
import urllib.request

class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, path=None, N=31, local=False):
     
        data = urllib.request.urlopen('http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt')

        aux=0
        ID=1

        for line in data:
            #print (line)
            line= str(line)
            if len(line.split(':'))==1:
                ID=ID+1
                aux=0
                self[int(ID/2)]={}
                
            else:
                aux=aux+1
                line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                line = line[:-5] #quitar la parte \r\n'
                line=float(line) #convertir de literal(y expresión científica) a número
                
                

                
                if aux == 1:
                    self[int(ID/2)]['HEALTH'] = line;
                    print(int(ID/2),aux,line)    

                if aux == 3:
                    self[ID/2]['TOA']    = line;
                        
                if aux == 4:
                    self[ID/2]['INC']    = line;
                        

                

                #line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                #line = line[:-5] #quitar la parte \r\n'
                #line=float(line) #convertir de literal(y expresión científica) a número
                #print(int(ID/2),line)

p1 = Yuma()

1 1 1.0
2 1 2.0
3 1 3.0
4 1 5.0
5 1 6.0
6 1 7.0
7 1 8.0
8 1 9.0
9 1 10.0
10 1 11.0
11 1 12.0
12 1 13.0
13 1 14.0
14 1 15.0
15 1 16.0
16 1 17.0
17 1 18.0
18 1 19.0
19 1 20.0
20 1 21.0
21 1 22.0
22 1 23.0
23 1 24.0
24 1 25.0
25 1 26.0
26 1 27.0
27 1 28.0
28 1 29.0
29 1 30.0
30 1 31.0
31 1 32.0


In [54]:
import urllib.request


class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, path=None, N=31, local=False):
     
        data = urllib.request.urlopen('http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt')

        aux=0

        for line in data:
            #print (line)
            line= str(line)
            if len(line.split(':'))==1:
                aux=0
                
            else:
                aux=aux+1
                line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                line = line[:-5] #quitar la parte \r\n'
                line=float(line) #convertir de literal(y expresión científica) a número
                   
                if aux == 1:
                    ID=line
                    self[int(ID)]={}
                    print(int(ID),aux,line)    
                    
                if aux == 2:
                    self[ID]['HEALTH']    = line;
                if aux == 3:
                    self[ID]['ECC']    = line;    
                if aux == 4:
                    self[ID]['TOA']    = line;                        
                if aux == 5:
                    self[ID]['INC']    = line;                        
                if aux == 6:
                    self[ID]['RRAND']    = line;                    
                if aux == 7:
                    self[ID]['SMA']    = line;                    
                if aux == 8:
                    self[ID]['RAND']    = line;                    
                if aux == 9:
                    self[ID]['AOP']    = line;                
                if aux == 10:
                    self[ID]['AM']    = line;                    
                if aux == 11:
                    self[ID]['AF0']    = line;                
                if aux == 12:
                    self[ID]['AF1']    = line;    
                if aux == 13:
                    self[ID]['AF1']    = line;
                

                #line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                #line = line[:-5] #quitar la parte \r\n'
                #line=float(line) #convertir de literal(y expresión científica) a número
                #print(int(ID/2),line)

p1 = Yuma()
print(p1[1]['TOA']) 


1 1 1.0
2 1 2.0
3 1 3.0
5 1 5.0
6 1 6.0
7 1 7.0
8 1 8.0
9 1 9.0
10 1 10.0
11 1 11.0
12 1 12.0
13 1 13.0
14 1 14.0
15 1 15.0
16 1 16.0
17 1 17.0
18 1 18.0
19 1 19.0
20 1 20.0
21 1 21.0
22 1 22.0
23 1 23.0
24 1 24.0
25 1 25.0
26 1 26.0
27 1 27.0
28 1 28.0
29 1 29.0
30 1 30.0
31 1 31.0
32 1 32.0
61440.0


In [134]:
import requests
import pandas as pd
import csv

url = 'http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt'
html = requests.get(url).content


reader = csv.reader(open(html), delimiter="\t")

#df_list = pd.read_html(html)
#df = df_list[-1]

print(html)
df.to_csv('my data.csv')

OSError: [Errno 36] File name too long: b'******** Week 1011 almanac for PRN-01 ********\r\nID:                         01\r\nHealth:                     000\r\nEccentricity:               0.8257865906E-002\r\nTime of Applicability(s):  61440.0000\r\nOrbital Inclination(rad):   0.9742300000\r\nRate of Right Ascen(r/s):  -0.7748894201E-008\r\nSQRT(A)  (m 1/2):           5153.620117\r\nRight Ascen at Week(rad):  -0.4721338949E+000\r\nArgument of Perigee(rad):   0.690804055\r\nMean Anom(rad):             0.8929457212E+000\r\nAf0(s):                    -0.1411437988E-003\r\nAf1(s/s):                  -0.7275957614E-011\r\nweek:                        1011\r\n\r\n******** Week 1011 almanac for PRN-02 ********\r\nID:                         02\r\nHealth:                     000\r\nEccentricity:               0.1872158051E-001\r\nTime of Applicability(s):  61440.0000\r\nOrbital Inclination(rad):   0.9532935591\r\nRate of Right Ascen(r/s):  -0.7954617056E-008\r\nSQRT(A)  (m 1/2):           515

In [98]:
class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, date, path=None, N=31, local=False):
        '''
        '''
        
        self.date = GPSDateTime(date)
        if not path:
            path = os.path.join(localpath, 'almanac')
        self.path = path
        if not os.path.exists(path):
            os.makedirs(path)
        for i in range(N):
            idate = self.date - timedelta(i)
            idoy = idate.doy
            fname = '%d%03d.ALM' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            fname = '%d%03d.alm' % (idate.year, idoy)
            if os.path.exists(os.path.join(path, fname)):
                break
            if not local:
                fname = self.download(idate)
                if fname:
                    break
            fname = False
        if not fname:
            raise RuntimeError('No almanac file found for %s in %s' \
                               % (self.date.date(), path))
        else:
            fid = open_file(os.path.join(path, fname))
            while True:
                try:
                    line = fid.next()
                except StopIteration:
                    break
                if not line: continue
                if line[:2]=='**':
                    ID = 'G%02d' % int(fid.next().split(':')[1])
                    self[ID]={}
                    self[ID]['HEALTH'] = float(fid.next().split(':')[1])
                    self[ID][' ']    = float(fid.next().split(':')[1])
                    self[ID]['TOA']    = float(fid.next().split(':')[1])
                    self[ID]['INC']    = float(fid.next().split(':')[1])
                    self[ID]['RRAND']  = float(fid.next().split(':')[1])
                    self[ID]['SMA']    = float(fid.next().split(':')[1])
                    self[ID]['RAND']   = float(fid.next().split(':')[1])
                    self[ID]['AOP']    = float(fid.next().split(':')[1])
                    self[ID]['AM']     = float(fid.next().split(':')[1])
                    self[ID]['AF0']    = float(fid.next().split(':')[1])
                    self[ID]['AF1']    = float(fid.next().split(':')[1])
                    week = int(fid.next().split(':')[1])
                    realweek = idate.GPSweek
                    if week<realweek:
                        week+=1024*(realweek/1024)
                    self[ID]['WK'] = week


In [ ]:
class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, path=None, N=31, local=False):

        
                    ID = 1
                    self[ID]={}
                    self[ID]['HEALTH'] = 12
                    self[ID][' ']    = 13
                    self[ID]['TOA']    = 14
                    self[ID]['INC']    = 15
                    self[ID]['RRAND']  = 16
                    self[ID]['SMA']    = 17
                    self[ID]['RAND']   = 18
                    self[ID]['AOP']    = 19
                    self[ID]['AM']     = 20
                    self[ID]['AF0']    = 21
                    self[ID]['AF1']    = 22
                    self[ID]['WK'] = 23
                            
                    ID = 2
                    self[ID]={}
                    self[ID]['HEALTH'] = 122
                    self[ID][' ']    = 132
                    self[ID]['TOA']    = 214
                    self[ID]['INC']    = 215
                    self[ID]['RRAND']  = 216
                    self[ID]['SMA']    = 217
                    self[ID]['RAND']   = 218
                    self[ID]['AOP']    = 219
                    self[ID]['AM']     = 202
                    self[ID]['AF0']    = 212
                    self[ID]['AF1']    = 222
                    self[ID]['WK'] = 232


In [ ]:
class MyClass:
  x = 5
p1 = Yuma()
print(p1[1]['HEALTH']) 
print(p1[2]['HEALTH']) 

In [84]:
import urllib.request
import math







GPS_GRAVITY_CONSTANT                       =  3.986005e14
GPS_CLOCK_CORRECTION_RELATIVISTIC_CONSTANT = -4.442807633e-10
GPS_WGS84_EARTH_ROTATION_RATE              =  7.2921151467e-05
LIGHT_SPEED                               =  299792458.0

SECONDS_IN_WEEK = 7*86400
    #TECUns          = 2.854       # TECU/ns according to GPS-Scinda, Charles Carrano, 4-7-08
TECUns        = 2.852       # TECU/ns according to TECalc_rinex, Pat Doherty, 2-21-94
F1              = 1.57542     # L1 Frequency (GHz)
F2              =1.22760     # L2 Frequency (GHz)
    
    
    
    
class Yuma(dict):
    '''
    Class to read an hold data from almanacs files (download if neccesary)
    in yuma format.
    Search almanac file for the given date (datetime) and path.
    '''
    def __init__(self, path=None, N=31, local=False):
     
        data = urllib.request.urlopen('http://celestrak.com/GPS/almanac/Yuma/2019/almanac.yuma.week1011.061440.txt')

        aux=0

        for line in data:
            #print (line)
            line= str(line)
            if len(line.split(':'))==1:
                aux=0
                
            else:
                aux=aux+1
                line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                line = line[:-5] #quitar la parte \r\n'
                line=float(line) #convertir de literal(y expresión científica) a número
                   
                if aux == 1:
                    ID=line
                    self[int(ID)]={}
                    print(int(ID),aux,line)    
                    
                if aux == 2:
                    self[ID]['HEALTH']    = line;
                if aux == 3:
                    self[ID]['ECC']    = line;    
                if aux == 4:
                    self[ID]['TOA']    = line;                        
                if aux == 5:
                    self[ID]['INC']    = line;                        
                if aux == 6:
                    self[ID]['RRAND']    = line;                    
                if aux == 7:
                    self[ID]['SMA']    = line;                    
                if aux == 8:
                    self[ID]['RAND']    = line;                    
                if aux == 9:
                    self[ID]['AOP']    = line;                
                if aux == 10:
                    self[ID]['AM']    = line;                    
                if aux == 11:
                    self[ID]['AF0']    = line;                
                if aux == 12:
                    self[ID]['AF1']    = line;
                if aux == 13:
                    self[ID]['WK']    = line;

                

                #line=line.split(':')[1].strip() #seleccionar la 2da parte y quitar los espacios
                #line = line[:-5] #quitar la parte \r\n'
                #line=float(line) #convertir de literal(y expresión científica) a número
                #print(int(ID/2),line)
                
    def calc_sat_pos(self, prn, dt, estimate_range):

        
        toe = self[prn]['TOA']
        m0  = self[prn]['AM']
        delta_n = 0
        tot = dt.GPSweek*SECONDS_IN_WEEK+dt.GPStow
        tk  = tot - (self[prn]['WK']*SECONDS_IN_WEEK+toe)
        # compute the corrected mean motion
        a   = self[prn]['SMA']**2
        n   = math.sqrt(GPS_GRAVITY_CONSTANT/(a*a*a))
        n  += delta_n
        # kepler's equation for eccentric anomaly
        M = m0 + n*tk
        E = M
        for i in range(8):
            E = M + self[prn]['ECC']*math.sin(E)

        cosE = math.cos(E)
        sinE = math.sin(E)
        # true anomaly
        v = math.atan2(math.sqrt(1.0-self[prn]['ECC']**2)*sinE, (cosE - self[prn]['ECC']))
        #argument of latitude
        u = v + self[prn]['AOP']
        # radius in orbital plane
        r = a * (1.0 - self[prn]['ECC']*math.cos(E))
        # orbital inclination
        i = self[prn]['INC']

        #argument of latitude correction
        #! no correction
        cosu = math.cos(u)
        sinu = math.sin(u)
        x_op = r*cosu
        y_op = r*sinu

        omegak = self[prn]['RAND'] + (self[prn]['RRAND'] - GPS_WGS84_EARTH_ROTATION_RATE)*tk - GPS_WGS84_EARTH_ROTATION_RATE*(toe + estimate_range/LIGHT_SPEED )

        # compute the WGS84 ECEF coordinates,
        # vector r with components x & y is now rotated using, R3(-omegak)*R1(-i)
        cos_omegak = math.cos(omegak)
        sin_omegak = math.sin(omegak)
        cosi = math.cos(i)
        sini = math.sin(i)
        x = x_op * cos_omegak - y_op * sin_omegak * cosi
        y = x_op * sin_omegak + y_op * cos_omegak * cosi
        z = y_op * sini

        return x,y,z
       

In [85]:
p1 = Yuma()
print(p1[1]['TOA']) 
print(p1.calc_sat_pos (15, GPSDateTime(2019,11,28), 1233344))

1 1 1.0
2 1 2.0
3 1 3.0
5 1 5.0
6 1 6.0
7 1 7.0
8 1 8.0
9 1 9.0
10 1 10.0
11 1 11.0
12 1 12.0
13 1 13.0
14 1 14.0
15 1 15.0
16 1 16.0
17 1 17.0
18 1 18.0
19 1 19.0
20 1 20.0
21 1 21.0
22 1 22.0
23 1 23.0
24 1 24.0
25 1 25.0
26 1 26.0
27 1 27.0
28 1 28.0
29 1 29.0
30 1 30.0
31 1 31.0
32 1 32.0
61440.0
(-7514478.241165271, 22003767.22539375, 12928428.574962296)
